# Pandas DataFrame Agent를 사용한 식품 보관 데이터 분석

이 노트북은 LangChain의 Pandas DataFrame Agent를 사용하여 `storageInfo.csv` 파일을 분석하고 시각화합니다.

In [ ]:
# 필요한 라이브러리 설치
# !pip install langchain langchain-openai langchain-experimental pandas matplotlib seaborn

In [ ]:
# 환경 설정
from dotenv import load_dotenv
import os

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import warnings
warnings.filterwarnings('ignore')

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)

## 1. 데이터 로드 및 기본 탐색

In [ ]:
# CSV 파일 로드
df = pd.read_csv("./data/storageInfo.csv")

print("📊 데이터셋 정보")
print("="*50)
print(f"• 전체 데이터 수: {len(df)}개")
print(f"• 컬럼 수: {len(df.columns)}개")
print(f"• 컬럼 목록: {', '.join(df.columns)}")
print(f"• 결측치: {df.isnull().sum().sum()}개")

In [ ]:
# 데이터 샘플 확인
print("\n📋 데이터 샘플 (상위 5개)")
df.head()

In [ ]:
# 데이터 타입 및 정보
df.info()

In [ ]:
# 기술 통계량
print("\n📈 보관일수(storageDays) 기술 통계")
df['storageDays'].describe()

## 2. Pandas DataFrame Agent 설정

In [ ]:
# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=2000
)

# Pandas DataFrame Agent 생성
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,  # 실행 과정을 볼 수 있도록 설정
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,  # 코드 실행 허용
    handle_parsing_errors=True
)

print("✅ Pandas DataFrame Agent가 성공적으로 생성되었습니다!")

## 3. 자연어 기반 데이터 분석

In [ ]:
# 분석 함수 정의
def analyze(question):
    """자연어 질문을 받아 데이터 분석 수행"""
    print(f"\n💡 질문: {question}")
    print("-" * 60)
    try:
        response = agent.run(question)
        print(f"📊 답변: {response}")
        return response
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

### 3.1 기본 분석

In [ ]:
# 카테고리 분석
analyze("카테고리별로 식품이 몇 개씩 있는지 알려주고, 가장 많은 카테고리는 무엇인가요?")

In [ ]:
# 보관일수 분석
analyze("평균 보관일수는 며칠이고, 가장 오래 보관 가능한 식품 TOP 5는 무엇인가요?")

In [ ]:
# 카테고리별 평균 보관일수
analyze("각 카테고리별 평균 보관일수를 계산하고, 내림차순으로 정렬해서 보여주세요.")

### 3.2 심화 분석

In [ ]:
# 보관 방법 분석
analyze("""
storageMethod 컬럼을 분석해서:
1. '냉장'이 포함된 식품은 몇 개인지
2. '냉동'이 포함된 식품은 몇 개인지
3. '실온'이 포함된 식품은 몇 개인지
각각 계산해주세요.
""")

In [ ]:
# 보관일수 구간 분석
analyze("""
보관일수를 다음 구간으로 나누어 분석해주세요:
- 1주 이하 (7일 이하)
- 1-2주 (8-14일)
- 2주-1개월 (15-30일)
- 1개월 이상 (30일 초과)
각 구간에 속하는 식품의 개수와 비율을 알려주세요.
""")

In [ ]:
# 과일 vs 야채 비교
analyze("""
과일과 야채 카테고리를 비교 분석해주세요:
1. 각각의 개수
2. 평균 보관일수
3. 최대/최소 보관일수
4. 어느 카테고리가 전반적으로 더 오래 보관 가능한지
""")

## 4. 데이터 시각화

In [ ]:
# 시각화를 위한 데이터 준비
category_counts = df['category'].value_counts()
category_avg_days = df.groupby('category')['storageDays'].mean().sort_values()
top10_storage = df.nlargest(10, 'storageDays')[['name', 'storageDays']]
bottom10_storage = df.nsmallest(10, 'storageDays')[['name', 'storageDays']]

In [ ]:
# 대시보드 스타일 시각화
fig = plt.figure(figsize=(16, 12))
fig.suptitle('🥬 식품 보관 데이터 분석 대시보드', fontsize=20, fontweight='bold', y=1.02)

# 1. 카테고리별 식품 개수
ax1 = plt.subplot(2, 3, 1)
colors = plt.cm.Set3(range(len(category_counts)))
bars = ax1.bar(category_counts.index, category_counts.values, color=colors)
ax1.set_title('카테고리별 식품 개수', fontsize=14, fontweight='bold')
ax1.set_xlabel('카테고리')
ax1.set_ylabel('개수')
ax1.tick_params(axis='x', rotation=45)
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom')

# 2. 카테고리별 평균 보관일수
ax2 = plt.subplot(2, 3, 2)
bars2 = ax2.barh(category_avg_days.index, category_avg_days.values, color='lightcoral')
ax2.set_title('카테고리별 평균 보관일수', fontsize=14, fontweight='bold')
ax2.set_xlabel('평균 보관일수 (일)')
for i, bar in enumerate(bars2):
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2., 
             f'{width:.1f}일', ha='left', va='center')

# 3. 보관일수 분포 히스토그램
ax3 = plt.subplot(2, 3, 3)
n, bins, patches = ax3.hist(df['storageDays'], bins=30, color='skyblue', 
                             edgecolor='black', alpha=0.7)
ax3.axvline(df['storageDays'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f'평균: {df["storageDays"].mean():.1f}일')
ax3.axvline(df['storageDays'].median(), color='green', linestyle='--', 
            linewidth=2, label=f'중앙값: {df["storageDays"].median():.1f}일')
ax3.set_title('보관일수 분포', fontsize=14, fontweight='bold')
ax3.set_xlabel('보관일수 (일)')
ax3.set_ylabel('빈도')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. 보관일수 TOP 10
ax4 = plt.subplot(2, 3, 4)
y_pos = np.arange(len(top10_storage))
bars4 = ax4.barh(y_pos, top10_storage['storageDays'].values, color='gold')
ax4.set_yticks(y_pos)
ax4.set_yticklabels(top10_storage['name'].values, fontsize=9)
ax4.set_title('보관일수 TOP 10 식품', fontsize=14, fontweight='bold')
ax4.set_xlabel('보관일수 (일)')
for i, bar in enumerate(bars4):
    width = bar.get_width()
    ax4.text(width, bar.get_y() + bar.get_height()/2., 
             f'{int(width)}일', ha='left', va='center', fontsize=8)

# 5. 카테고리별 박스플롯
ax5 = plt.subplot(2, 3, 5)
categories = df['category'].unique()
data_to_plot = [df[df['category'] == cat]['storageDays'].values for cat in categories]
bp = ax5.boxplot(data_to_plot, labels=categories, patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
ax5.set_title('카테고리별 보관일수 분포', fontsize=14, fontweight='bold')
ax5.set_xlabel('카테고리')
ax5.set_ylabel('보관일수 (일)')
ax5.tick_params(axis='x', rotation=45)
ax5.grid(True, alpha=0.3)

# 6. 보관일수 구간별 파이차트
ax6 = plt.subplot(2, 3, 6)
bins = [0, 7, 14, 30, 60, 200]
labels = ['1주 이하\n(≤7일)', '1-2주\n(8-14일)', '2주-1달\n(15-30일)', 
          '1-2달\n(31-60일)', '2달 이상\n(>60일)']
df['storage_range'] = pd.cut(df['storageDays'], bins=bins, labels=labels)
storage_range_counts = df['storage_range'].value_counts()
colors_pie = plt.cm.Pastel1(range(len(storage_range_counts)))
wedges, texts, autotexts = ax6.pie(storage_range_counts.values, 
                                     labels=storage_range_counts.index,
                                     autopct='%1.1f%%',
                                     colors=colors_pie,
                                     startangle=90)
ax6.set_title('보관일수 구간별 비율', fontsize=14, fontweight='bold')
for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.savefig('storage_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ 대시보드가 'storage_dashboard.png' 파일로 저장되었습니다.")

## 5. 보관 방법 텍스트 분석

In [ ]:
# 보관 방법 키워드 분석
keywords = ['냉장', '냉동', '실온', '서늘한', '밀폐', '신문지', '랩', '비닐', '키친타월']
keyword_counts = {}

for keyword in keywords:
    count = df['storageMethod'].str.contains(keyword, na=False).sum()
    keyword_counts[keyword] = count

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 키워드 빈도 막대 그래프
sorted_keywords = dict(sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True))
ax1.bar(sorted_keywords.keys(), sorted_keywords.values(), color='teal', alpha=0.8)
ax1.set_title('보관 방법 주요 키워드 빈도', fontsize=14, fontweight='bold')
ax1.set_xlabel('키워드')
ax1.set_ylabel('언급 횟수')
ax1.tick_params(axis='x', rotation=45)
for i, (key, value) in enumerate(sorted_keywords.items()):
    ax1.text(i, value, str(value), ha='center', va='bottom')

# 보관 방법 유형 분류
storage_types = {
    '냉장 보관': df['storageMethod'].str.contains('냉장', na=False).sum(),
    '냉동 보관': df['storageMethod'].str.contains('냉동', na=False).sum(),
    '실온 보관': df['storageMethod'].str.contains('실온|서늘한', na=False).sum(),
}

ax2.pie(storage_types.values(), labels=storage_types.keys(), autopct='%1.1f%%',
        colors=['lightblue', 'lightcoral', 'lightgreen'], startangle=90)
ax2.set_title('주요 보관 방법 분류', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. 인사이트 도출

In [ ]:
# Agent를 통한 인사이트 도출
print("\n🔍 데이터 기반 인사이트")
print("="*60)

insights = analyze("""
전체 데이터를 종합적으로 분석하여 다음 질문에 답해주세요:

1. 가장 주목할 만한 패턴 3가지는 무엇인가요?
2. 일반 가정에서 실용적으로 활용할 수 있는 식품 보관 팁 3가지를 제시해주세요.
3. 카테고리별로 특별히 주의해야 할 점은 무엇인가요?
""")

In [ ]:
# 통계 요약 테이블 생성
summary_stats = df.groupby('category').agg({
    'name': 'count',
    'storageDays': ['mean', 'median', 'min', 'max', 'std']
}).round(1)

summary_stats.columns = ['개수', '평균일수', '중앙값', '최소일수', '최대일수', '표준편차']
summary_stats = summary_stats.sort_values('평균일수', ascending=False)

print("\n📊 카테고리별 통계 요약")
print("="*60)
print(summary_stats.to_string())

## 7. 실용적 활용 방안

In [ ]:
# 보관 기간별 식품 리스트 생성
print("\n📝 보관 기간별 식품 관리 가이드")
print("="*60)

# 단기 보관 식품 (7일 이하)
short_term = df[df['storageDays'] <= 7][['name', 'storageDays', 'storageMethod']]
print("\n🔴 단기 보관 식품 (1주일 이내 소비 필요)")
print("-" * 40)
for _, row in short_term.head(10).iterrows():
    print(f"• {row['name']}: {row['storageDays']}일")

# 중기 보관 식품 (8-30일)
mid_term = df[(df['storageDays'] > 7) & (df['storageDays'] <= 30)][['name', 'storageDays']]
print("\n🟡 중기 보관 식품 (1주~1달)")
print("-" * 40)
for _, row in mid_term.head(10).iterrows():
    print(f"• {row['name']}: {row['storageDays']}일")

# 장기 보관 식품 (30일 초과)
long_term = df[df['storageDays'] > 30][['name', 'storageDays']]
print("\n🟢 장기 보관 식품 (1달 이상)")
print("-" * 40)
for _, row in long_term.head(10).iterrows():
    print(f"• {row['name']}: {row['storageDays']}일")

In [ ]:
# 최종 요약
print("\n📌 분석 결과 요약")
print("="*60)
print(f"• 총 분석 식품 수: {len(df)}개")
print(f"• 평균 보관 기간: {df['storageDays'].mean():.1f}일")
print(f"• 가장 많은 카테고리: {category_counts.index[0]} ({category_counts.values[0]}개)")
print(f"• 가장 오래 보관 가능: {df.loc[df['storageDays'].idxmax(), 'name']} ({df['storageDays'].max()}일)")
print(f"• 가장 짧게 보관: {df.loc[df['storageDays'].idxmin(), 'name']} ({df['storageDays'].min()}일)")
print("\n✨ 분석이 완료되었습니다!")